In [1]:
import logging
import uuid

from random import sample

from scripts.utils.file_utils import load_json, dump_json
from scripts.ripe_atlas.ping import PING
from scripts.ripe_atlas.atlas_api import get_measurements_from_tag
from default import ANCHORS_FILE, PROBES_FILE, TARGET_ALL_VP, MEASUREMENT_CONFIG_PATH, RIPE_CREDENTIALS


logging.basicConfig(level=logging.INFO)


NB_TARGETS = 2
NB_VPS = 4

#NBTARGETS = -1
#NB_VPS = -1

In [2]:
# load datasets

anchors = load_json(ANCHORS_FILE)

probes = load_json(PROBES_FILE)

## select targets and vps dataset

In [3]:
if NB_TARGETS > 0:
    targets = sample(anchors, k=NB_TARGETS)
else:
    targets = anchors

for i in range(len(targets)):
    targets[i] = targets[i]["address_v4"]


probes.extend(anchors)

vps = {}

if NB_VPS > 0:
    shuffle = sample(probes, k=NB_VPS)
    for probe in shuffle:
        vps[probe["address_v4"]] = probe
else:
    shuffle = sample(probes)
    for probe in shuffle:
        vps[probe["address_v4"]] = probe


print(f"nb targets: {len(targets)}")
print(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 4


# Tier 2: probing each target

## measure targets

In [4]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes and all anchors as vps",
    "type": "target",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
dump_json(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)


try:
    cached_results = load_json(TARGET_ALL_VP)
    logging.info(
        f"initial length targets: {len(targets)}, cached measurements : {len(cached_results)}"
    )

    targets = list(
        set(targets).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(targets)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(targets)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_target(
    targets=targets,
    vps=vps,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
dump_json(measurement_config, config_file)

INFO:root:No cached results available
INFO:root:Starting measurements 84bc3179-0ee0-40cd-a09c-456a5e29db62 with parameters: dry_run=True; nb_targets=2; nb_vps=4.
INFO:root:measurement : 84bc3179-0ee0-40cd-a09c-456a5e29db62 done


## retrieve traget probing results

In [2]:
#measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))
measurement_results = get_measurements_from_tag("425a780c-6f08-4ced-9372-48a8db976986")
# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_json(measurement_results, TARGET_ALL_VP)

nb measurements retrieved: 2
91.234.194.103 : 4
185.142.156.40 : 4
